In [5]:
EXC = 'EXC'
MSG = 'MSG'

LEVEL_THRESHOLD_MAPPING = {EXC: 0.5, 
                           MSG: 0.5}
PRIORITIES  = [EXC, MSG]

In [4]:
import collections
import os
import numpy as np
import matplotlib.pyplot as plt

class HierarchyClustering(object):
    def __init__(self, name_featmat_mapping):
        self.route_clusters_mapping = {}
        self.route_level_mapping = {}
        self.filter_featmat_mapping = {PRIORITIES[i]: name_featmat_mapping[i] for i in range(len(PRIORITIES))}
        self.filter_threshold_mapping = LEVEL_THRESHOLD_MAPPING
        
    def fit(self):
        self.traverse()
        self.create_utility_mapping()
        self.calculate_CDF()
        
    def traverse(self):
# =============================================================================
#         Traverse the whole branches
# =============================================================================
        level = 0
        root_mat = self.filter_featmat_mapping[PRIORITIES[level]]
        doc_ids = [doc_id for doc_id in range(root_mat.shape[0])]
        idxmat_docid_mapping = dict(zip(doc_ids, doc_ids))
        docid_idxmat_mapping = idxmat_docid_mapping.copy()
        clusterid_docids_mapping = self.run_cluster(self.filter_featmat_mapping[PRIORITIES[level]], self.filter_threshold_mapping[PRIORITIES[level]], idxmat_docid_mapping, docid_idxmat_mapping)
        self.traverse_tree_structure_by_recursion(str(level), level, idxmat_docid_mapping, clusterid_docids_mapping)
    
    def create_utility_mapping(self):
# =============================================================================
#         # Route - Level map
# =============================================================================
        self.route_level_mapping = {}
        for idx, (route, clusters) in enumerate(self.route_clusters_mapping.items()):
            if idx == 0:
                self.route_level_mapping[route] = 0
            for idx, clusterid in enumerate(clusters):
                child_route = route + '->' + str(idx)
                self.route_level_mapping[child_route] = child_route.count('->')
                
        # Level - Routes Map
        self.level_routes_mapping = {}
        for route, level in self.route_level_mapping.items():
            self.level_routes_mapping[level] = self.level_routes_mapping.get(level, []) + [route]
            
    def build_submatrix(self, docids, feature_matrix):
# =============================================================================
#         Build submatrix
# =============================================================================
        mat_size = len(docids)
        idxmat_docid_mapping = dict(zip(range(mat_size), docids))
        docid_idxmat_mapping = dict(zip(docids, range(mat_size)))
        submatrix = np.zeros((mat_size, mat_size))
        for i in range(mat_size):
            for j in range(i, mat_size):
                submatrix[i][j] = feature_matrix[idxmat_docid_mapping[i]][idxmat_docid_mapping[j]]
                submatrix[j][i] = feature_matrix[idxmat_docid_mapping[j]][idxmat_docid_mapping[i]]
        return submatrix, idxmat_docid_mapping, docid_idxmat_mapping
    
    def traverse_tree_structure_by_recursion(self, route, level, idxmat_docid_mapping, clusterid_docids_mapping):
# =============================================================================
#         Traverse tree structure by using recursion
# =============================================================================
        self.route_clusters_mapping[route] = list(clusterid_docids_mapping.values())
        self.route_level_mapping[route] = level
        if level < len(PRIORITIES) - 1 :
            for idx, (cluster, docids) in enumerate(clusterid_docids_mapping.items()):
                submatrix, r_idxmat_docid_mapping, r_docid_idxmat_mapping = self.build_submatrix(docids, self.filter_featmat_mapping[PRIORITIES[level + 1]])
                r_clusterid_docids_mapping = self.run_cluster(submatrix, self.filter_threshold_mapping[PRIORITIES[level + 1]], r_idxmat_docid_mapping, r_docid_idxmat_mapping)
                route = route + '->' + str(idx)
                self.traverse_tree_structure_by_recursion(route, level+1, r_idxmat_docid_mapping, r_clusterid_docids_mapping)
                route = route[:route.rfind('->')]
        else:
            for i, sublist in enumerate(list(clusterid_docids_mapping.values())):
                route = route + '->' +str(i)
                self.route_clusters_mapping[route] =[[i for i in sublist]]
                route = route[:route.rfind('->')]
                
    def run_cluster(self, feature_matrix, threshold, idxmat_docid_mapping, docid_idxmat_mapping):
# =============================================================================
#         Cluster
# =============================================================================
        
        def order_subclusters(clusterid_docids_mapping):
# =============================================================================
#             Order
# =============================================================================
            
            if len(clusterid_docids_mapping.keys()) > 1:
                clusterids = list(clusterid_docids_mapping.keys())
                clusterid_numelements_mapping = {clusterid: len(docids) for clusterid, docids in clusterid_docids_mapping.items()}
                cluster_most_docid = sorted(clusterid_numelements_mapping.items(), key=lambda x:x[1], reverse=True)[0][0]
                
                # Initial step: by default I take the pivot pointing to the index 0, while clusterid_most_docid is moved to index 0
                stacked_clusterids = clusterids.copy()
                stacked_clusterids.remove(cluster_most_docid)
                stacked_clusterids.insert(0, cluster_most_docid)
                
                pivot_index = 0
                while pivot_index < len(clusterids):
                    # Updating pivot pointing to the next cluster after reordering clusterid which has similarity score 
                    # in decending order. 
                    pivot = stacked_clusterids[pivot_index]
                    rep_docid_of_pivot_cluster = clusterid_docids_mapping[pivot][0]       #choose the first docid to be represented information of that clusterid
                    sub_list = stacked_clusterids[pivot_index + 1:]                       #take the left elements to compare 
                    if len(sub_list) > 0:
                        max_score = -1
                        max_index = 0
                        for idx, clusterid in enumerate(sub_list):
                            rep_docid_of_cluster_x = clusterid_docids_mapping[clusterid][0]
                            similarity_of_2_clusters = feature_matrix[docid_idxmat_mapping[rep_docid_of_pivot_cluster]][docid_idxmat_mapping[rep_docid_of_cluster_x]]
                            if max_score < similarity_of_2_clusters:
                                max_score = similarity_of_2_clusters
                                max_index = idx
    
                        next_pivot = sub_list[max_index]
                        stacked_clusterids.remove(next_pivot)
                        stacked_clusterids.insert(0, next_pivot)
                    else:
                        last_element = stacked_clusterids.pop()
                        stacked_clusterids.insert(0, last_element)
                    pivot_index += 1
                
                clusterid_neworder_mapping = {ordered_clusterid: idx for idx, ordered_clusterid in enumerate(stacked_clusterids)}
                clusterid_docids_mapping = {k: clusterid_docids_mapping[v] for k, v in clusterid_neworder_mapping.items()}
                clusterid_docids_mapping = dict(collections.OrderedDict(sorted(clusterid_docids_mapping.items(), key=lambda x:x[0])))
            return clusterid_docids_mapping
                
        
        docid_idxmat_mapping = dict(zip(idxmat_docid_mapping.values(), idxmat_docid_mapping.keys()))
        doc_ids = list(idxmat_docid_mapping.values())
        clusterid = 0
        clusterid_docids_mapping = {}
        while len(doc_ids) > 0:
            pivot = doc_ids[0]
            sub = [doc_id for doc_id in doc_ids if doc_id != pivot]
            clusterid_docids_mapping[clusterid] = [pivot]
            doc_ids.remove(pivot)
            for doc_id in sub:
                if feature_matrix[docid_idxmat_mapping[pivot]][docid_idxmat_mapping[doc_id]] >= threshold:
                    clusterid_docids_mapping[clusterid].append(doc_id)
                    doc_ids.remove(doc_id)
            clusterid += 1
        clusterid_docids_mapping = order_subclusters(clusterid_docids_mapping)
        return clusterid_docids_mapping

    def export_graphviz(self, graphviz_path):
# =============================================================================
#          Draw tree graph by Graphviz
#          Example input: os.environ["PATH"] += os.pathsep + 'C:\\Users\\quanthuynh\\temp\\graphviz-2.38\\release\\bin\\'
# =============================================================================
        
        depth = 1
        try:
            os.environ["PATH"] += os.pathsep + graphviz_path
            from graphviz import Digraph
            dot = Digraph(comment='Hierarchy Tree Structure Clustering')
            
            for level, routes in self.level_routes_mapping.items():
                if level <= depth + 1 and level != 0:
                    for route in routes:
                        node_label = PRIORITIES[level - 1] + ' ' + route[route.rfind('->') + 2:]
                        dot.node(route, node_label)
                            
            for level, routes in self.level_routes_mapping.items():
                if level <= depth and level < len(PRIORITIES):
                    if level == 0:
                        for child_route in self.level_routes_mapping[level + 1]:
                            dot.edge('root', child_route)
                    else:
                        for route in routes:
                            children_routes = [route + '->' + str(i) for i in self.route_clusters_mapping[route].keys()]
                            for child_route in children_routes:
                                dot.edge(route, child_route)
            dot.render('graphviz/tree-structure.gv', view=True)
        except:
            # Cannot import graphviz
            pass
    
    def calculate_CDF(self):
        self.level_CDF_mapping = {}
        for level_threshold in range(len(PRIORITIES) + 1):
            route_elements_mapping = {route: sum(len(errorids) for errorids in clusters) for route, clusters in self.route_clusters_mapping.items() if self.route_level_mapping[route]==level_threshold}
            route_elements_mapping = collections.OrderedDict({route: elements for route, elements in sorted(route_elements_mapping.items(), key=lambda x: x[1])})
            CDF = HierarchyClustering.get_CDF(route_elements_mapping)
            colors = HierarchyClustering.get_color_based_occurrence_ratio(route_elements_mapping)
            self.route_docids_mapping = {route: [item for sublist in self.route_clusters_mapping[route] for item in sublist] for route in route_elements_mapping.keys() if self.route_level_mapping[route]==level_threshold}
            self.level_CDF_mapping[level_threshold] = (self.route_docids_mapping, colors, CDF)
        return self.level_CDF_mapping 
            
    def plot_CDF(self, level_threshold):
        route_elements_mapping = {route: sum(len(errorids) for errorids in clusters.values()) for route, clusters in self.route_clusters_mapping.items() if self.route_level_mapping[route]==level_threshold}
        HierarchyClustering.plot_cluster_topic_histogram(route_elements_mapping)
        
    @staticmethod
    def get_info_of_cluster_id(cluster_id, cluster_dict, cumulative_pro_func):
        total = sum(cluster_dict.values())
        percent = int(cluster_dict[cluster_id] / total * 100 )
        cum_per = cumulative_pro_func[cluster_id]
        text = 'ID {}\n{}%'.format(cluster_id, percent, cum_per)
        return text

    @staticmethod
    def get_CDF(route_elements_mapping):
        CDF = {}
        account = 0
        total = sum(route_elements_mapping.values())
        for clusterid, numelements in route_elements_mapping.items():
            CDF[clusterid] = int((account + numelements) / total * 100)
            account = account + numelements
        return CDF

    @staticmethod
    def get_color_based_occurrence_ratio(route_elements_mapping):
        color_range = [10,20,70]
        color = []
        for k, v in route_elements_mapping.items():
            if v < color_range[0]:
                color.append('red')
            elif v > color_range[1]:
                color.append('lightgreen')
            else:
                color.append('yellow')
        return color
        
    @staticmethod
    def plot_cluster_topic_histogram(utility_dictionary):
        keys = utility_dictionary.keys()
        values = utility_dictionary.values()
        cluster_dict = dict(zip(keys, values))
        cluster_dict = {k: v for k, v in sorted(cluster_dict.items(), key=lambda x: x[1])}
        cumulative_pro_func = HierarchyClustering.get_CDF(cluster_dict)
        keys = [HierarchyClustering.get_info_of_cluster_id(x, cluster_dict, cumulative_pro_func) for x in cluster_dict.keys()]
        values = list(cluster_dict.values())
        color = HierarchyClustering.get_color_based_occurrence_ratio(cumulative_pro_func)
    
        # Plot 
        fig=plt.figure(figsize=(20, 16), dpi= 80, facecolor='w', edgecolor='k')
        ax1 = fig.add_subplot(111)
        ax1.bar(keys, values, color = color)
        plt.title('Failed Execution Test Result (document) distribution per Topic Clustering')
        plt.xlabel('rare <---------------------------------------------> usual', fontsize=12)
        plt.ylabel('Number of failed execution test result', fontsize=12)
        ax1.grid(True)
    
        # Plot step function for cumulative probablity distribution function
        ax2 = ax1.twinx()
        x_step = [i for i in range(len(keys))]
        y_step = [v for k, v in cumulative_pro_func.items()]
        ax2.plot(x_step, y_step, color = 'b')
        ax2.set_ylabel('Accumulative percentage',fontsize=12)
    
        # Plot annotation in the diagram
        annotations = [str(cumulative_pro_func[x]) for x in cluster_dict]
        for i, txt in enumerate(annotations):
            plt.annotate(txt, (x_step[i], y_step[i]))
        plt.show()
        return cluster_dict, color, cumulative_pro_func